In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import (
    get_kelly, filter_leverage, get_cumulative_returns, backtest
)

In [3]:
kospi = pd.read_csv('KOSPI200.csv', index_col='Date')
kospi = kospi.iloc[::-1].reset_index(drop=True)

returns_kospi = kospi['Price'].pct_change()
shift_returns_kospi = returns_kospi.shift(-1)

expanding_kelly = get_kelly(returns_kospi)

parameters_df = pd.DataFrame()
parameters_df['kelly'] = expanding_kelly
parameters_df['Next day return'] = shift_returns_kospi
parameters_df['mean'] = returns_kospi.expanding(400).mean() 